In [52]:
import math
import numpy as np
import polars as pl


In [53]:
def depreciate_value(purchase_price, month, decay_rate, residual_percentage):
    """Value and monthly depreciation using exponential decay: \( v(t) = p e^{-k t} \), steeper for new cars."""
    k = decay_rate
    
    t = (month - 0.5) / 12.0  # Mid-month time for approximation
    value = purchase_price * math.exp(-k * t)
    
    # Floor at residual
    residual = purchase_price * residual_percentage
    value = max(value, residual)
    
    # Monthly depreciation: approximate rate during the month
    monthly_depr = (k / 12.0) * value
    
    return value, monthly_depr

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
/var/folders/z0/bm_l503s7r14l8hm2r_1p3380000gn/T/ipykernel_7436/3676916455.py:2: SyntaxWarning: invalid escape sequence '\('
  """Value and monthly depreciation using exponential decay: \( v(t) = p e^{-k t} \), steeper for new cars."""


In [54]:
data = {"id": [1, 2], 
        "name": ["tesla_model_3", "opel_corsa_e"], 
        "type": ["buy", "buy"], 
        "build_year": [2019, 2022],
        "build_month": [1, 1],
        "buy_year": [2025, 2025],
        "buy_month": [11, 11],
        "purchase_cost": [18000.0, 17000.0],
        "road_taxes_yearly": [1000.0, 700.0],
        "insurance_monthly": [280.0, 180.0],
        "fuel_per_km": [0.08, 0.08],
        "depreciation_k": [0.08, 0.08]}
df = pl.DataFrame(data)

In [57]:
def cost_over_time(row: dict, n_years: int, n_kilometer_per_year: int) -> np.ndarray:
    n_months = n_years * 12
    costs = np.zeros(n_months)

    monthly_costs = (row["road_taxes_yearly"] / 12) + row["insurance_monthly"] + (n_kilometer_per_year / 12 * row["fuel_per_km"])

    for i in range(n_months):
        car_age_months = (row["buy_year"] * 12 + row["buy_month"] + i + 1) - (row["build_year"] * 12 + row["build_month"])
        if (row["type"] == "buy") and i == 0:
            costs[i] += row["purchase_cost"]
        if i >= 1:
            costs[i] += costs[i-1]
        remaining_value, depreciation_monthly = depreciate_value(purchase_price=row["purchase_cost"], month=car_age_months, decay_rate=row["depreciation_k"], residual_percentage=0.2)
        costs[i] += (monthly_costs + depreciation_monthly)
        if i == (n_months - 1):
            costs[i] -= remaining_value
        costs[i] = np.round(costs[i], decimals=2)

    return costs

def simulate_costs_for_fleet(df: pl.DataFrame, n_years: int, n_kilometer_per_year) -> dict[int, np.ndarray]:
    results = {}
    for row in df.iter_rows(named=True):
        car_id = row["id"]
        results[car_id] = {}
        results[car_id]["total_costs_over_time"] = cost_over_time(row, n_years, n_kilometer_per_year)
        results[car_id]["name"] = row["name"]
        
    return results

costs_dict = simulate_costs_for_fleet(df, n_years=5, n_kilometer_per_year=16000)
costs_dict

{1: {'total_costs_over_time': array([18539.23, 19078.  , 19616.32, 20154.18, 20691.59, 21228.55,
         21765.07, 22301.15, 22836.79, 23371.99, 23906.76, 24441.1 ,
         24975.01, 25508.5 , 26041.56, 26574.21, 27106.44, 27638.26,
         28169.67, 28700.67, 29231.26, 29761.45, 30291.24, 30820.63,
         31349.63, 31878.24, 32406.46, 32934.29, 33461.73, 33988.79,
         34515.47, 35041.78, 35567.71, 36093.27, 36618.46, 37143.29,
         37667.75, 38191.85, 38715.59, 39238.97, 39762.  , 40284.68,
         40807.01, 41328.99, 41850.62, 42371.91, 42892.86, 43413.47,
         43933.74, 44453.68, 44973.29, 45492.57, 46011.52, 46530.15,
         47048.45, 47566.43, 48084.09, 48601.44, 49118.47, 42627.29]),
  'name': 'tesla_model_3'},
 2: {'total_costs_over_time': array([17428.12, 17855.69, 18282.71, 18709.19, 19135.13, 19560.53,
         19985.39, 20409.72, 20833.53, 21256.81, 21679.57, 22101.82,
         22523.55, 22944.77, 23365.49, 23785.7 , 24205.41, 24624.63,
         25043.35

In [49]:
chart =  (
    df.plot.point(
        x="id",
        y="purchase_cost",
    )
    .properties(width=500, title="Costs")
    .configure_scale(zero=False)
    .configure_axisX(tickMinStep=1)
)
chart.encoding.x.title = "ID"
chart.encoding.y.title = "Cost"
chart

alt.Chart(...)